In [1]:
import os
import sys

import django

sys.path.append('../')  # add path to project root dir

os.environ["DJANGO_SETTINGS_MODULE"] = "flourish.settings"

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

django.setup()

  * Reading config from flourish.ini
Loading Data Encryption (init)...
 * loading keys from /Users/nimrodmunatsi/source/flourish-code-space/flourish/crypto_fields
 * loading rsa.restricted.public ... Done.
 * loading rsa.restricted.private ... Done.
 * loading rsa.local.public ... Done.
 * loading rsa.local.private ... Done.
 * loading aes.local ... Done.
 * loading aes.restricted ... Done.
 * loading salt.local ... Done.
 * loading salt.restricted ... Done.
 Done loading Data Encryption (init)...


/Users/nimrodmunatsi/.venv/flourish/lib/python3.9/site-packages/django_q/conf.py:139: UserWarning: Retry and timeout are misconfigured. Set retry larger than timeout, 
        failure to do so will cause the tasks to be retriggered before completion. 
        See https://django-q.readthedocs.io/en/latest/configure.html#retry for details.
  warn(
/Users/nimrodmunatsi/.venv/flourish/lib/python3.9/site-packages/simple_history/models.py:164: UserWarning: HistoricalRecords added to abstract model (HistoryManagerMixin) without inherit=True
  warnings.warn(msg, UserWarning)
/Users/nimrodmunatsi/.venv/flourish/lib/python3.9/site-packages/simple_history/models.py:164: UserWarning: HistoricalRecords added to abstract model (ScheduleModelMixin) without inherit=True
  warnings.warn(msg, UserWarning)
/Users/nimrodmunatsi/.venv/flourish/lib/python3.9/site-packages/simple_history/models.py:164: UserWarning: HistoricalRecords added to abstract model (VisitModelMixin) without inherit=True
  warnings.wa

Loading Data Encryption ...
 * found encryption keys in /Users/nimrodmunatsi/source/flourish-code-space/flourish/crypto_fields.
 * using model django_crypto_fields.crypt.
 Done loading Data Encryption.
Loading Edc Consent ...
 * checking for site consents ...
 * registered consents 'consents' from 'flourish_caregiver'
 * registered consents 'consents' from 'flourish_child'
 * registered consents 'consents' from 'pre_flourish'
 * registered consents 'consents' from 'flourish_facet'
 * flourish_caregiver.subjectconsent 1 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_caregiver.subjectconsent 2 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_caregiver.subjectconsent 3 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_caregiver.tbadolconsent 1 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_caregiver.tbinformedconsent 1 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_child.childdummysubjectconsent 1 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_child.ch

In [2]:
import json
from tqdm import tqdm
from flourish_caregiver.models import CaregiverChildConsent, ScreeningPregWomen, \
    ScreeningPregWomenInline

# Open the file in read mode
with open('screeningpregwomen.json', 'r') as file:
    # Read the content of the file
    content = file.read()
    # Parse the JSON content to a Python object (list)
    all_screenings = json.loads(content)

# Evaluate the string to convert it into a list
for screening in tqdm(all_screenings):
    child_subject_identifier = None
    try:
        consent = CaregiverChildConsent.objects.filter(
            subject_consent__screening_identifier=screening.get('fields').get('screening_identifier')).latest(
            'consent_datetime')
    except CaregiverChildConsent.DoesNotExist:
        pass
    else:
        if consent and not consent.study_child_identifier:
            child_subject_identifier = consent.subject_identifier

    options = {
        'report_datetime': screening.get('fields').get('report_datetime'),
        'hiv_testing': screening.get('fields').get('hiv_testing'),
        'breastfeed_intent': screening.get('fields').get('breastfeed_intent'),
        'ineligibility': screening.get('fields').get('ineligibility'),
        'is_eligible': screening.get('fields').get('is_eligible'),
        'is_consented': screening.get('fields').get('is_consented'),
        'child_subject_identifier': child_subject_identifier if child_subject_identifier else None,
    }
    
    try: 
        ScreeningPregWomen.objects.get(pk=screening.get('pk'))
    except ScreeningPregWomen.DoesNotExist:
        pass
    else:
        screening_inline_obj, _ = ScreeningPregWomenInline.objects.update_or_create(
            mother_screening_id=screening.get('pk'), **options)
        screening_inline_obj.save()

file.close()

100%|██████████| 305/305 [00:13<00:00, 22.41it/s]
